To use the RNN model,we need a lot of sequence. To do so, because we have only one serie to predict, We have to cut our input in multiple sequence. We can choose sequence of one month. 
Indeed, the input of RNN have this dimension : (n_squence, n_observation, n_feature)
The length of the sequence will have an impact on the seasonality capturing. 

The units in RNN add feature which store the context from previous input

We have to tried RNN,LSTM, GRU and maybe take a look at the bidirectional RNN

In [2]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU,Dense
from sklearn.model_selection import train_test_split
from data_import import import_data, import_clean_energy_data,import_clean_weather_data
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

In [3]:
df=import_clean_energy_data()

Importing energy_dataset data from /home/alan/code/MarvinLiebisch/forecasting-electricity-prices/raw_data/energy_dataset.csv...


In [55]:
#df.sort_index(ascending=False,inplace=True)

In [5]:
y=df[['price_day_ahead']]
X=df.drop(columns=['price_day_ahead','price_actual'])

In [6]:
def create_sequences(X, y, observation_length,step):
    n_SEQUENCES=len(range(0,len(X) - observation_length - 24,step))
    X_sequenced = np.zeros((n_SEQUENCES, observation_length, X.shape[1])) # initialize the dataframe with the new sequences
    y_sequenced= np.zeros((n_SEQUENCES, step,1))
    index=0
    for i in range(len(X) - observation_length - 24,0,-step):
        X_sequenced[index] = X.iloc[i:i + observation_length].values
        y_sequenced[index]= y.iloc[i + observation_length:i + observation_length + 24].values
        index=index+1
    return X_sequenced, y_sequenced
step=24
observation_length = 24 * 7  # For example, a week of data for the sequence
X_sequenced, y_sequenced = create_sequences(X, y, observation_length,step)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_sequenced, y_sequenced, test_size=0.2, random_state=42, shuffle=True)

In [121]:
model = Sequential()
model.add(LSTM(units=64, input_shape=X_train.shape[1:], return_sequences=True))
model.add(LSTM(units=64))
model.add(Dense(24))  # Output layer with 24 neurons (one for each hour)

# Compile and train the model
model.compile(optimizer='rmsprop', loss='mse',metrics=['mae'])
history =model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.3,verbose=1)

Epoch 1/100
51/51 [==============================] - 94s 869ms/step - loss: 2800.2568 - mae: 51.2158 - val_loss: 1352.5197 - val_mae: 33.9939
Epoch 2/100
51/51 [==============================] - 47s 925ms/step - loss: 2417.1741 - mae: 47.3499 - val_loss: 1134.5916 - val_mae: 30.7552
Epoch 3/100
51/51 [==============================] - 44s 862ms/step - loss: 2113.3994 - mae: 44.0772 - val_loss: 945.1246 - val_mae: 27.7240
Epoch 4/100
51/51 [==============================] - 26s 509ms/step - loss: 1835.8331 - mae: 40.8814 - val_loss: 777.1118 - val_mae: 24.8103
Epoch 5/100
51/51 [==============================] - 27s 525ms/step - loss: 1583.1598 - mae: 37.7022 - val_loss: 631.7353 - val_mae: 22.0450
Epoch 6/100
51/51 [==============================] - 29s 567ms/step - loss: 1349.7284 - mae: 34.5605 - val_loss: 507.6353 - val_mae: 19.4677
Epoch 7/100
29/51 [================>.............] - ETA: 14s - loss: 1147.3169 - mae: 31.5553

KeyboardInterrupt: 

In [123]:
def plt_history(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    plt.show()